In [1]:
#leyendo texto 1 de prueba
#este en realidad ya no sirve pero lo dejo como evidencia de que se hizo prueba con este texto

with open('Academic/nf0069.txt') as f:
    contents=f.read()

## código nuevo

In [43]:
##Abriendo corpus
import glob

lista_files = glob.glob('Academic/*.txt')
corpus = []

for file in lista_files:
    with open(file, 'r', encoding='utf8') as f:
        corpus+=[f.read()]

In [44]:
## número de textos

len(corpus)

8

In [45]:
corpus[0][:250]

"nf0069: when talking about the French gangster film very often and \nparadoxically in French one talks about the policier that's the name of the \ngenre so when you you see the word policier er I-E police which relates to the \npolice it doesn't necessa"

In [46]:
#importando librerias
import spacy
import nltk
import string

In [47]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pauba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
#eliminar () y []
def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [49]:
## aplicamos la función "a" a cada texto del corpus

corpus = [a(c) for c in corpus]

In [50]:
corpus[0][:250]

"nf0069: when talking about the French gangster film very often and \nparadoxically in French one talks about the policier that's the name of the \ngenre so when you you see the word policier er I-E police which relates to the \npolice it doesn't necessa"

In [51]:
#dividir por salto de linea (cada texto)
corpus = [c.replace('♪','').split('\n') for c in corpus]

In [52]:
corpus[0][:5]

['nf0069: when talking about the French gangster film very often and ',
 "paradoxically in French one talks about the policier that's the name of the ",
 'genre so when you you see the word policier er I-E police which relates to the ',
 "police it doesn't necessarily mean that it's to do with a police film or police ",
 "procedure as the you have in the American genres but it's co-, covers all er "]

In [53]:
#eliminamos strings vacíos (cada texto)

corpus = [[sentence.strip() for sentence in c if len(sentence)>0] for c in corpus]

In [54]:
# volvemos a juntar las oraciones :)

corpus = [' '.join(c) for c in corpus]

In [55]:
corpus[0][:250]

"nf0069: when talking about the French gangster film very often and paradoxically in French one talks about the policier that's the name of the genre so when you you see the word policier er I-E police which relates to the police it doesn't necessaril"

In [56]:
#dividir en oraciones, importar la funcion
from nltk import sent_tokenize

In [57]:
#dividir en oraciones cada texto
sentence_list = []
for texto in corpus:
    sentences = sent_tokenize(texto)
    sentence_list+=[sentences]

In [58]:
len(sentence_list)

8

In [59]:
sentence_list[0][:5]

["nf0069: when talking about the French gangster film very often and paradoxically in French one talks about the policier that's the name of the genre so when you you see the word policier er I-E police which relates to the police it doesn't necessarily mean that it's to do with a police film or police procedure as the you have in the American genres but it's co-, covers all er virtually all the films er to do with crime er pretty much from the nineteen- fifties onwards so er a gangster film and a police film will both be called policier in French and sometimes you'll see the in slang this becomes polar er i'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep sf0070: sorry nf0069: okay we're just beginning polar is the slang word for policier so le polar refers to the genre of the police crime gangster movie in French er specifically from the nineteen-fifties late forties fifties onwards and so that i will talk today about a particular genre or subgenre of Fre

In [60]:
#numero de oraciones por texto

for c in sentence_list:
    print(len(c))

1
1
1
1
1
1
1
1


In [61]:
#### los datos de youtube tienen dos formas:

## corpus: lista de strings de largo 24
## sentence_list: lista de listas de strings, que indican las oraciones de cada texto

## análisis!

### 1. Nominalizaciones
El trabajo aquí es con listas de oraciones. Extraemos los sustantivos :)

In [62]:
!pip install spacy
!spacy download en

In [63]:
oraciones_nlp = []

In [64]:
##TAGGEANDO EL LEMMA Y EL POS
import spacy
nlp = spacy.load("en_core_web_sm")

for sentence_text in sentence_list:
    if len(sentence_text)>1:
        sent = []
        for sentence in sentence_text:
            doc = nlp(sentence)
            sent += [[(token.text.lower(),token.pos_) for token in doc]]
        oraciones_nlp += [sent]

In [65]:
len(sentence_list)

8

In [66]:
len(oraciones_nlp)

0

In [67]:
def number_tokens_types(oraciones):
    number = []
    for oracion in oraciones:
        number+=[pair[0] for pair in oracion]
    return len(number),len(set(number))

In [68]:
## número de tokens y types para los 23 textos (dejé fuera el que era una sola oración)

for oraciones in oraciones_nlp:
    print(number_tokens_types(oraciones))

In [69]:
def number_nouns(oraciones):
    number = []
    for oracion in oraciones:
        number+=[pair for pair in oracion if pair[1]=='NOUN']
    return len(number),len(set(number))

In [70]:
## número de nouns tokens y types para los 23 textos (dejé fuera el que era una sola oración)

for oraciones in oraciones_nlp:
    print(number_nouns(oraciones))

In [71]:
#reglas de nominalization
no_nom = ['thing', 'things', 'somethings', 'something', 'anything', 'everything', 'nothing','original', 'special', 'normal', 'version', 'tutorial', 'moment']
terminacion = ['ment','ments','tions', 'tion','sions', 'sion', 'ibility','ibilities', 'ity','ities', 'ness','nesses', 'al','als','ings', 'ing'] 

In [72]:
def nominalization(oraciones):
    nom = []
    for oracion in oraciones:
        for pair in oracion:
            if pair[0] not in no_nom:
                if pair[1]=='NOUN':
                    for END in terminacion:
                        if pair[0].endswith(END):
                            nom+=[pair[0]]
    return nom

In [73]:
for oraciones in oraciones_nlp:
    print(nominalization(oraciones))

In [74]:
#promedio oraciones
len(sentences)/24

0.041666666666666664

## Palabras académicas

In [75]:
#abriendo la lista de palabras academicas
with open('AcademicWordList.txt') as f:
    AWL=f.read()

In [76]:
#dividir por salto de linea
AWL= AWL.split('\n')

In [77]:
##palabras uncias

for palabra in AWL:
    if palabra not in AWL:
        AWL += [palabra]

In [78]:
##eliminar espacios en blanco
AWL = [palabra for palabra in AWL if len(palabra)>0]
AWL[:10]

['analysis',
 'approach',
 'area',
 'assessment',
 'assume',
 'authority',
 'available',
 'benefit',
 'concept',
 'consistent']

In [79]:
AWL_dic={palabra:0 for palabra in AWL}

In [80]:
##SEPARAR POR PALABRAS

words=[]
for text in corpus:
    for word in text.split(' '):
        words+=[word]
words[:15]

['nf0069:',
 'when',
 'talking',
 'about',
 'the',
 'French',
 'gangster',
 'film',
 'very',
 'often',
 'and',
 'paradoxically',
 'in',
 'French',
 'one']

In [81]:
for palabra in words:
    if palabra in AWL:    
        AWL_dic[palabra]+=1
    

In [82]:
AWL_dic

{'analysis': 7,
 'approach': 6,
 'area': 17,
 'assessment': 0,
 'assume': 8,
 'authority': 4,
 'available': 5,
 'benefit': 5,
 'concept': 0,
 'consistent': 0,
 'constitutional': 0,
 'context': 34,
 'contract': 3,
 'create': 7,
 'data': 1,
 'definition': 2,
 'derived': 0,
 'distribution': 9,
 'economic': 67,
 'environment': 7,
 'established': 2,
 'estimate': 0,
 'evidence': 0,
 'export': 13,
 'factors': 15,
 'financial': 9,
 'formula': 0,
 'function': 3,
 'identified': 1,
 'income': 9,
 'indicate': 0,
 'individual': 11,
 'interpretation': 0,
 'involved': 13,
 'issues': 21,
 'labour': 107,
 'legal': 4,
 'legislation': 3,
 'major': 35,
 'method': 0,
 'occur': 2,
 'percent': 0,
 'period': 42,
 'policy': 47,
 'principle': 23,
 '\x0cprocedure': 0,
 'process': 10,
 'required': 5,
 'research': 1,
 'response': 4,
 'role': 15,
 'section': 3,
 'sector': 17,
 'significant': 7,
 'similar': 11,
 'source': 5,
 'specific': 7,
 'structure': 10,
 'theory': 55,
 'variables': 0,
 'achieve': 6,
 'acquisiti

In [83]:
values = sum(AWL_dic.values())
print(values)

2047


In [84]:
values/len(words)*100

3.0455417850713404

## código viejo

In [2]:
##Abriendo corpus
import glob
lista_files = glob.glob('Academic/*.txt')
corpus = []

for file in lista_files:
     with open(file, 'r') as file_input:
         corpus+=[file_input.read()]

In [3]:
##quitar cosas () y []
##crear funcion
def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [4]:
##quitar los () y []
corpus = a(corpus)

In [5]:
#dividir por salto de linea
corpus= corpus.split('\n')

In [6]:
corpus[:15]

['nf0069: when talking about the French gangster film very often and ',
 "paradoxically in French one talks about the policier that's the name of the ",
 'genre so when you you see the word policier er I-E police which relates to the ',
 "police it doesn't necessarily mean that it's to do with a police film or police ",
 "procedure as the you have in the American genres but it's co-, covers all er ",
 'virtually all the films er to do with crime er pretty much from the nineteen-',
 'fifties onwards so er a gangster film and a police film will both be called ',
 "policier in French and sometimes you'll see the in slang this becomes polar er ",
 "i'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep ",
 'sf0070: sorry ',
 "nf0069: okay we're just beginning polar is the slang word for policier so le ",
 'polar refers to the genre of the police crime gangster movie in French er ',
 'specifically from the nineteen-fifties late forties fifties onwards and so that ',

In [7]:
len(corpus)

5032

In [8]:
#dividir en oraciones, importar la funcion
from nltk import sent_tokenize

#dividir en oraciones
sentences = []
for sentence in corpus:
    sent_token = sent_tokenize(sentence)
    sentences+=[sent_token]

In [9]:
#número de oraciones
len(sentences)

5032

In [10]:
len(sentences)/8

629.0

In [11]:
#reglas de nominalization
no_nom = 'thing', 'things', 'somethings', 'something', 'anything', 'everything', 'nothing','original', 'special', 'normal', 'version'
terminacion = 'ment','ments','tions', 'tion','sions', 'sion', 'ibility','ibilities', 'ity','ities', 'ness','nesses', 'al','als','ings', 'ing' 
s='involvement'
s.endswith(terminacion)

True

In [12]:
#oraciones con POSIBLES nominalizaciones
nominalization = []
for oracion in sentences:
    for word in oracion:
        if word.endswith(terminacion):
            nominalization +=[word]
len(nominalization)

358

In [13]:
#10% para analizar
nominalization[:32]

['that film er but i just wanted to show you er the cover that is the original',
 "reason why er i'm talking about the er the Serie Noire writers er in relation",
 'to the films is is that they are on the one hand those indirect connections',
 'and his er er body movements are very different from what you see in something',
 "nf0069: what's interesting er about this er well many things are interesting",
 'the exception of Bob le Flambeur which was very much an independent production',
 'where censorship was much less er intent on questions of morals of sexual',
 "those films we're looking at this week are very important as a transitional",
 "sm0072: well 'cause i've never been candidate so er because because staying",
 'with the war and the German occupation of France er this is the a reading',
 'quote unquote of contemporary French society a society which was er moving',
 "years away from General de Gaulle's Fifth Republic and and the real",
 "realist films er of community of workers 

In [14]:
##porcentaje nominalizaciones
len(nominalization)/len(sentences)*100

7.114467408585055

In [15]:
#aplanar la lista de strings
flat_sentences = []
for item in sentences:
        for word in item:
            flat_sentences+=[word]

In [16]:
#remover puntuación
import re

def remove_punctuation(s):
    return re.sub(r'[^\w\s]','',s)

In [17]:
flat_sentences_nopunc =[]

for sent in flat_sentences:
    flat_sentences_nopunc+=[remove_punctuation(sent)]

In [18]:
flat_sentences_nopunc[:15]

['nf0069 when talking about the French gangster film very often and',
 'paradoxically in French one talks about the policier thats the name of the',
 'genre so when you you see the word policier er IE police which relates to the',
 'police it doesnt necessarily mean that its to do with a police film or police',
 'procedure as the you have in the American genres but its co covers all er',
 'virtually all the films er to do with crime er pretty much from the nineteen',
 'fifties onwards so er a gangster film and a police film will both be called',
 'policier in French and sometimes youll see the in slang this becomes polar er',
 'im sorry i hope its not too awkward if i move er polar is POLAR er is yep',
 'sf0070 sorry',
 'nf0069 okay were just beginning polar is the slang word for policier so le',
 'polar refers to the genre of the police crime gangster movie in French er',
 'specifically from the nineteenfifties late forties fifties onwards and so that',
 'i will',
 'talk today about a

In [19]:
##SEPARAR POR PALABRAS
words=[]
for sentence in flat_sentences_nopunc:
    for word in sentence.split(' '):
        words+=[word]
words[:15]

['nf0069',
 'when',
 'talking',
 'about',
 'the',
 'French',
 'gangster',
 'film',
 'very',
 'often',
 'and',
 'paradoxically',
 'in',
 'French',
 'one']

In [20]:
len(words)

67236

In [21]:
##eliminar lista de palabras que no sirve, como los codigos de dialogo y "er"
stopwords = ['er', 'nf0069', 'nf1167', 'nm0063', 'nm0067', 'nm0969', 'nm1123', 'nm1166', 'nm1222']
for word in list(words):
    if word in stopwords:
        words.remove(word)

In [22]:
##eliminar espacios en blanco
words = [palabra for palabra in words if len(palabra)>0]

In [23]:
words[:15]

['when',
 'talking',
 'about',
 'the',
 'French',
 'gangster',
 'film',
 'very',
 'often',
 'and',
 'paradoxically',
 'in',
 'French',
 'one',
 'talks']

In [24]:
len(words)

64738

In [25]:
#promedio palabra por oración

len(words)/len(sentences)

12.865262321144675

In [26]:
#promedio palabra por texto
len(words)/8

8092.25

In [27]:
##palabras uncias

types = []
for palabra in words:
    if palabra not in types:
        types += [palabra]

In [28]:
types[:15]

['when',
 'talking',
 'about',
 'the',
 'French',
 'gangster',
 'film',
 'very',
 'often',
 'and',
 'paradoxically',
 'in',
 'one',
 'talks',
 'policier']

In [29]:
len(types)

5964

In [30]:
len(types)/len(words)*100

9.212518150081868

In [31]:
#descagando spacy
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [32]:
##TAGGEANDO EL LEMMA Y EL POS
nouns=[]
nlp = spacy.load("en_core_web_sm")
for oracion in flat_sentences_nopunc:
    doc = nlp(oracion)

    for token in doc:
        ##print(token.text, token.lemma_, token.pos_, token.tag_)
        if token.pos_ =="NOUN":
            nouns+=[token.text]
nouns[:15]

In [33]:
#reglas de nominalization
no_nom = 'thing', 'things', 'somethings', 'something', 'anything', 'everything', 'nothing', 'original', 'special', 'normal', 'version'
terminacion = 'ment','ments','tions', 'tion','sions', 'sion', 'ibility','ibilities', 'ity','ities', 'ness','nesses', 'al','als','ings', 'ing'
s='involvement'
s.endswith(terminacion)

True

In [34]:
#oraciones con POSIBLES nominalizaciones
nominalization = []
for oracion in sentences:
     for word in oracion:
        nom = False 
        for term in terminacion: 
                if word.endswith(term):
                    nom = True 
                    if nom==True and word not in no_nom: 
                            nominalization +=[word]
len(nominalization)   

359

In [35]:
#10% de nominalicaciones para analizar
nominalization[:36]

['that film er but i just wanted to show you er the cover that is the original',
 "reason why er i'm talking about the er the Serie Noire writers er in relation",
 'to the films is is that they are on the one hand those indirect connections',
 'and his er er body movements are very different from what you see in something',
 "nf0069: what's interesting er about this er well many things are interesting",
 'the exception of Bob le Flambeur which was very much an independent production',
 'where censorship was much less er intent on questions of morals of sexual',
 "those films we're looking at this week are very important as a transitional",
 "sm0072: well 'cause i've never been candidate so er because because staying",
 'with the war and the German occupation of France er this is the a reading',
 'quote unquote of contemporary French society a society which was er moving',
 "years away from General de Gaulle's Fifth Republic and and the real",
 "realist films er of community of workers 

In [54]:
import spacy

In [55]:
##TAGGEANDO EL LEMMA Y EL POS
nouns=[]
nlp = spacy.load("en_core_web_sm")
for oracion in flat_sentences_nopunc:
    doc = nlp(oracion)

    for token in doc:
        ##print(token.text, token.lemma_, token.pos_, token.tag_)
        if token.pos_ =="NOUN":
            nouns+=[token.text]
nouns[:15]

['gangster',
 'film',
 'talks',
 'name',
 'genre',
 'word',
 'police',
 'police',
 'police',
 'film',
 'police',
 'procedure',
 'genres',
 'co',
 'films']

In [56]:
#lista POSIBLES nominalizaciones, solo palabras
nom1= []
for word in words:
    for term in terminacion:
        if word.endswith(term):
            if word not in no_nom:
                if word not in nom1:
                    nom1+=[word]

In [57]:
nom1[:15]

['talking',
 'beginning',
 'going',
 'hybridity',
 'starting',
 'Obsession',
 'lettering',
 'sexuality',
 'during',
 'notion',
 'subverting',
 'translation',
 'several',
 'telling',
 'publishing']

In [58]:
len(nom1)

1075

In [59]:
#abriendo la lista de palabras academicas
with open('AcademicWordList.txt') as f:
    AWL=f.read()

In [60]:
#dividir por salto de linea
AWL= AWL.split('\n')

In [61]:
##palabras uncias

for palabra in AWL:
    if palabra not in AWL:
        AWL += [palabra]

In [62]:
AWL[:10]

['analysis',
 'approach',
 'area',
 'assessment',
 'assume',
 'authority',
 'available',
 'benefit',
 'concept',
 'consistent']

In [63]:
##eliminar espacios en blanco
AWL = [palabra for palabra in AWL if len(palabra)>0]
AWL[:10]

['analysis',
 'approach',
 'area',
 'assessment',
 'assume',
 'authority',
 'available',
 'benefit',
 'concept',
 'consistent']

In [64]:
AWL_dic={palabra:0 for palabra in AWL}

In [65]:
for palabra in words:
    if palabra in AWL:    
        AWL_dic[palabra]+=1

In [66]:
AWL_dic

{'analysis': 7,
 'approach': 6,
 'area': 17,
 'assessment': 0,
 'assume': 8,
 'authority': 4,
 'available': 5,
 'benefit': 5,
 'concept': 0,
 'consistent': 0,
 'constitutional': 0,
 'context': 34,
 'contract': 3,
 'create': 7,
 'data': 1,
 'definition': 2,
 'derived': 0,
 'distribution': 9,
 'economic': 67,
 'environment': 7,
 'established': 2,
 'estimate': 0,
 'evidence': 0,
 'export': 13,
 'factors': 15,
 'financial': 9,
 'formula': 0,
 'function': 3,
 'identified': 1,
 'income': 9,
 'indicate': 0,
 'individual': 11,
 'interpretation': 0,
 'involved': 13,
 'issues': 21,
 'labour': 107,
 'legal': 4,
 'legislation': 3,
 'major': 35,
 'method': 0,
 'occur': 2,
 'percent': 0,
 'period': 42,
 'policy': 47,
 'principle': 23,
 '\x0cprocedure': 0,
 'process': 10,
 'required': 5,
 'research': 1,
 'response': 4,
 'role': 15,
 'section': 3,
 'sector': 17,
 'significant': 7,
 'similar': 11,
 'source': 5,
 'specific': 7,
 'structure': 10,
 'theory': 55,
 'variables': 0,
 'achieve': 6,
 'acquisiti

In [67]:
values = sum(AWL_dic.values())
print(values)

2043


In [68]:
values/len(words)*100

3.1557972133831753

In [69]:
#lista POSIBLES nominalizaciones, solo palabras
nom1= []
for word in words:
    for term in terminacion:
        if word.endswith(term):
            if word not in no_nom:
                if word not in nom1:
                    if word in nouns:
                        nom1+=[word]
len(nom1)

585

In [70]:
len(nom1)/len(words)*100

0.9036423738762397